# Графики

Мы знаем, что графики можно рисовать в питоне с помощью Matplotlib. Но когда мы пишем сайт, удобнее использовать готовые инструменты, специально предназначенные для браузера, например, [D3.js](https://d3js.org/) или [Google Charts](https://developers.google.com/chart/).

Мы поближе познакомимся с Google Charts. Рассмотрим пример [отсюда](https://developers.google.com/chart/interactive/docs/quick_start):

In [ ]:
<html>
  <head>
    <!--Load the AJAX API-->
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
    <script type="text/javascript">

      // Load the Visualization API and the corechart package.
      // The first argument to google.charts.load is the version name or number, as a string. 
      // If you specify 'current', this causes the latest official release of Google Charts to be loaded. 
      google.charts.load('current', {'packages':['corechart'], 'language': 'ru'});
      
      // corechart contains: bar, column, line, area, stepped area, bubble, pie, donut, combo, candlestick, histogram, scatter
      // you can add other packages as needed e.g., {packages: ['corechart', 'table', 'sankey']}
      
      // Locales are used to customize text for a country or language, 
      // affecting the formatting of values such as currencies, dates, and numbers.
      // By default, the Google Charts is loaded with the "en" locale. 

      // Set a callback to run when the Google Visualization API is loaded.
      google.charts.setOnLoadCallback(drawChart);

      // Callback that creates and populates a data table,
      // instantiates the pie chart, passes in the data and
      // draws it.
      function drawChart() {

        // Create the data table.
        var data = new google.visualization.DataTable();
        data.addColumn('string', 'Topping');
        data.addColumn('number', 'Slices');
        data.addRows([
          ['Mushrooms', 3],
          ['Onions', 1],
          ['Olives', 1],
          ['Zucchini', 1],
          ['Pepperoni', 2]
        ]);

        // Set chart options
        var options = {'title':'How Much Pizza I Ate Last Night',
                       'width':400,
                       'height':300};

        // Instantiate and draw our chart, passing in some options.
        var chart = new google.visualization.PieChart(document.getElementById('chart_div'));
        chart.draw(data, options);
      }
    </script>
  </head>

  <body>
    <!--Div that will hold the pie chart-->
    <div id="chart_div"></div>
  </body>
</html>

Гугл-графики рисуют картинку за нас, а нам лишь нужно написать в html-странице сами данные. А передавать в html-шаблон данные с помощью фласка мы как раз умеем.

Данные нужно передать в JavaScript класс DataTable - это двумерная табличка. У каждой колонки должен быть тип (string, number etc), опционально - label и id.

Существует много типов графиков, и у каждого есть документация, полностью его описывающая. Перед рисованием конкретной картинки, нужно изучить ее документацию, потому что:
  * каждый тип графика ожидает разный формат DataTable,
  * у разных графиков могут быть разные options,
  * для некоторых графиков нужно подключать отдельный пакет, как тут: {packages: ['corechart', 'table']}.

# Flask + Charts

Вспомним наше приложение, которое получало текст и возвращало его же, но каждому слову приписывалась часть речи. 
Отредактируем это приложение так, чтобы мы могли посмотреть в столбчатой диаграмме распределение частей речи.

In [ ]:
from flask import Flask
from flask import render_template, request
from pymystem3 import Mystem
from collections import defaultdict  # добавили defaultdict

m = Mystem()
app = Flask(__name__)


def add_POS(text):
    result = ''
    ana = m.analyze(text)
    d = defaultdict(int)  # добавили defaultdict
    for i in ana:
        result += i['text']
        if i['text'].strip() and 'analysis' in i and i['analysis']:
            pos = i['analysis'][0]['gr'].split('=')[0].split(',')[0]
            d[pos] += 1  # собираем информацию о встречаемости частей речи
            result += '<span class="pos">{}</span>'.format(pos)
    return result, d

@app.route('/pos', methods=['get', 'post'])
def pos_text():
    if request.form:
        text = request.form['text']
        result, d = add_POS(text)  # не забыли отредактировать результат здесь
        result = result.replace('\n', '<br>')
        return render_template('mystem_page.html', input=text, text=result, data=d)  # передаем новые данные в функцию
    return render_template('mystem_page.html', data={}) # передаем пустую переменную в функцию


if __name__ == '__main__':
    app.run(debug=True)

А вот так будет выглядеть html-шаблон:

In [ ]:
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Части речи слов</title>
	<style>
		.pos {vertical-align: sub; color: blue; font-size: 70%;}
		.result {font-size: 24px;}
	</style>
    
    
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
    <script type="text/javascript">
      google.charts.load('current', {'packages':['corechart'], 'language': 'ru'});
      google.charts.setOnLoadCallback(drawChart);
      function drawChart() {
        var data = new google.visualization.DataTable();
        data.addColumn('string', 'Часть речи');
        data.addColumn('number', 'Количество');
        data.addRows([
          {% for pos, num in data.items() %}
              ['{{ pos }}', {{ num }}],
          {% endfor %}
        ]);

        var options = {'title':'Части речи',
                       'width':600,
                       'height':300,
                        'legend': { position: "none" }};

        var chart = new google.visualization.BarChart(document.getElementById('chart_div'));
        chart.draw(data, options);
      }
    </script>
    
    
    
</head>
<body>
<a href="/">На главную.</a>
	<form method='post'>
		<h1>Ваш текст:</h1>
		<textarea rows="4" cols="50" name="text">{{ input }}</textarea>
		<br>
		<input type="submit" value="Разобрать">
	</form>
	{% if text %}
        <div id="chart_div"></div>
		<h1>Разбор:</h1>
		<p class="result"> {{ text | safe}}</p>
	{% endif %}

</body>
</html>

Действующий пример находится здесь - http://2017learnpython.pythonanywhere.com/pos.

Google Charts позволяет легко рисовать даже сложные графики, например, [диаграммы Гантта](https://developers.google.com/chart/interactive/docs/gallery/ganttchart) или [географические карты](https://developers.google.com/chart/interactive/docs/gallery/geochart).

# Задания на семинар

1) Создайте html-страничку с [диаграммой Гантта](https://developers.google.com/chart/interactive/docs/gallery/ganttchart) с этапами работы над каким-либо вашим проектом (курсовой, дз по какому-то предмету, ваш будущий стартап и т.д.). (Это руками)

2) Скачать список пользователей, состоящих в группе https://vk.com/hse_university (VK API). Нарисовать [столбчатую диаграмму](https://developers.google.com/chart/interactive/docs/gallery/columnchart), показывающую распределение пользователей по их году рождения.

3) Нарисуйте [Calendar Chart](https://developers.google.com/chart/interactive/docs/gallery/calendar ), показывающую количество твитов в каждый день в [аккаунте сколково](https://github.com/elmiram/2017learnpython/blob/master/data/skolkovo_ru.csv).